In [1]:
import yaml
import cv2
import os
import numpy as np
import pandas as pd
import zipfile
import requests
from collections import Counter

## **Data Augmentation**

### Horizontal Flipping

In [20]:
def Vertical_Flipping(image_files, train_images_dir, save_dir):
    flipped_images = []  
    saved_filenames = []  

    for file_name in image_files:  # Loop through each image file
        img_path = os.path.join(train_images_dir, file_name)  # Get full path to the image
        img = cv2.imread(img_path)  
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  

        # Flip over y-axis (horizontal flip)
        flipped_img = cv2.flip(img, 1)  # 1 means horizontal flip
        flipped_images.append(flipped_img)  # Save flipped image in the list

        if save_dir: 
            save_path = os.path.join(save_dir, f"flipped_{file_name}")  
            flipped_bgr = cv2.cvtColor(flipped_img, cv2.COLOR_RGB2BGR)  # Convert back to BGR for saving
            cv2.imwrite(save_path, flipped_bgr)  # Save the flipped image

            save_name = f"flipped_{file_name}"  # Name of the saved file
            save_path = os.path.join(save_dir, save_name)  
            cv2.imwrite(save_path, flipped_bgr)  
            
            saved_filenames.append(save_name)  # Add filename to the list

    return flipped_images, saved_filenames  # Return flipped images and their filenames


In [70]:
train_flipped_images_dir = "C:\\Users\\jorge\\Documents\\CV\\CSWK\\african-wildlife\\train\\Flipped_images"
flipped_images, saved_Flipped_filenames = Vertical_Flipping(image_files, train_images_dir, train_flipped_images_dir)

### Brightness Adjustment

In [15]:
def adjust_brightness_manual(image, brightness):

    img = image.copy()
    height, width, channels = img.shape

    for i in range(height):
        for j in range(width):
            for k in range(channels):
                pix = img[i, j, k] + brightness  
                img[i, j, k] = min(max(pix, 0), 255)  # Clamp between 0 and 255

    return img.astype(np.uint8)

In [31]:
train_images_dir = "C:\\Users\\jorge\\Documents\\CV\\CSWK\\african-wildlife\\train\\images"
output_dir = "C:\\Users\\jorge\\Documents\\CV\\CSWK\\african-wildlife\\train\\darker_images"

for file_name in image_files:
    file_path = os.path.join(train_images_dir, file_name)
    
    # Cargar imagen
    original_img = cv2.imread(file_path)
    
    if original_img is None:
        print(f"Error loading {file_name}")
        continue

    brighter_img = adjust_brightness_manual(original_img, brightness=-60)

    output_file_name = "Darker_" + file_name
    output_path = os.path.join(output_dir, output_file_name)

    cv2.imwrite(output_path, brighter_img)

print("Imágenes con brillo ajustado guardadas correctamente.")

Imágenes con brillo ajustado guardadas correctamente.


In [174]:
original_labels_dir = r"C:\Users\jorge\Documents\CV\CSWK\african-wildlife\valid\labels" 
new_images_dir = r"C:\Users\jorge\Documents\CV\CSWK\african-wildlife\valid\Flipped_images"  
new_labels_dir = r"C:\Users\jorge\Documents\CV\CSWK\african-wildlife\valid\Flipped_labels"  

if not os.path.exists(new_labels_dir):  
    os.makedirs(new_labels_dir)  # Creates the folder

for filename in os.listdir(new_images_dir):  # Loops through each flipped image file
    if filename.lower().endswith(('.jpg', '.jpeg', '.png')):  
        original_name = filename.replace("flipped_", "")  # Removes 'flipped_' prefix to find the original image name
        label_name = os.path.splitext(original_name)[0] + ".txt"  # Get the corresponding original label filename

        original_label_path = os.path.join(original_labels_dir, label_name)  # Full path to original label
        new_label_name = os.path.splitext(filename)[0] + ".txt"  # New label filename for flipped image
        new_label_path = os.path.join(new_labels_dir, new_label_name)  # Full path to save new label

        if os.path.exists(original_label_path):  # Check if the original label file exists
            shutil.copyfile(original_label_path, new_label_path)  # Copy the original label to the new label path
        else:
            print(f"No se encontró label para: {filename}")  # Warning if label not found

In [37]:
original_labels_dir = "C:\\Users\\jorge\\Documents\\CV\\CSWK\\african-wildlife\\train\\labels"  
flipped_labels_dir = "C:\\Users\\jorge\\Documents\\CV\\CSWK\\african-wildlife\\train\\labels_flipped"  
flipped_images_dir = "C:\\Users\\jorge\\Documents\\CV\\CSWK\\african-wildlife\\train\\Flipped_images" 

os.makedirs(flipped_labels_dir, exist_ok=True)  # Create output folder if it doesn't exist

# Get list of flipped images
image_files = [f for f in os.listdir(flipped_images_dir) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]

for image_file in image_files:
    base_name = os.path.splitext(image_file)[0]  
    original_name = base_name.replace("flipped_", "")  # Gets original name → e.g., 1 (4)
    label_file = original_name + ".txt"  
    original_label_path = os.path.join(original_labels_dir, label_file)  

    if not os.path.exists(original_label_path):  # Checks if the label exists
        print(f"Label original no encontrada para: {image_file}")  
        continue 

    # Read and process the original label
    with open(original_label_path, "r") as f:
        lines = f.readlines()

    flipped_lines = [] 
    for line in lines:
        parts = line.strip().split()  # Split the line into parts
        if len(parts) != 5:  # Skip badly formatted lines
            continue

        class_id, x_center, y_center, width, height = parts  # Unpack YOLO label values
        flipped_x = 1.0 - float(x_center)  # Flip x_center horizontally
        flipped_line = f"{class_id} {flipped_x:.6f} {y_center} {width} {height}\n"  # Format new label line
        flipped_lines.append(flipped_line)  # Add to list

    # Save the new flipped label with the same name as the flipped image
    flipped_label_path = os.path.join(flipped_labels_dir, base_name + ".txt")
    with open(flipped_label_path, "w") as f:
        f.writelines(flipped_lines)

print("Labels reflejadas creadas para imágenes flippeadas.")

Labels reflejadas creadas para imágenes flippeadas.


In [39]:
brighter_labels_dir = "C:\\Users\\jorge\\Documents\\CV\\CSWK\\african-wildlife\\train\\brighter_labels"  
darker_labels_dir = "C:\\Users\\jorge\\Documents\\CV\\CSWK\\african-wildlife\\train\\darker_labels"

# Create folders if they don't exist
os.makedirs(brighter_labels_dir, exist_ok=True)  
os.makedirs(darker_labels_dir, exist_ok=True) 

# Process each label file
for label_file in os.listdir(original_labels_dir):  # Loops through label files
    if not label_file.endswith(".txt"):  # Skip non-txt files
        continue

    original_path = os.path.join(original_labels_dir, label_file)  # Full path to original label

    brighter_name = "Brighter_" + label_file  # New filename for brighter image label
    darker_name = "Darker_" + label_file  # New filename for darker image label

    brighter_path = os.path.join(brighter_labels_dir, brighter_name)  # Destination path for brighter label
    darker_path = os.path.join(darker_labels_dir, darker_name)  # Destination path for darker label

    shutil.copyfile(original_path, brighter_path)  # Copy original label to brighter label folder
    shutil.copyfile(original_path, darker_path)  # Copy original label to darker label folder

print("Labels copiados y renombrados para imágenes más brillantes y más oscuras.")  # Confirmation message


Labels copiados y renombrados para imágenes más brillantes y más oscuras.
